In [4]:
from qiskit.providers.ibmq import IBMQ
from qiskit.providers.aer import AerSimulator

provider = IBMQ.get_provider(hub="ibm-q-community", group="mit-hackathon", project="main")
backend = provider.get_backend('ibm_oslo')

# generate a simulator that mimics the real quantum system with the latest calibration results
backend_sim = AerSimulator.from_backend(backend)

/opt/conda/lib/python3.8/site-packages/qiskit_aer/noise/device/models.py:365: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (256641.33307868752 > 2 * 86159.68087196324). Truncating to maximum value.
  warn("Device model returned an invalid T_2 relaxation time greater than"
/opt/conda/lib/python3.8/site-packages/qiskit_aer/noise/device/models.py:365: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (256641.33307868752 > 2 * 86159.68087196324). Truncating to maximum value.
  warn("Device model returned an invalid T_2 relaxation time greater than"
/opt/conda/lib/python3.8/site-packages/qiskit_aer/noise/device/models.py:365: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (256641.33307868752 > 2 * 86159.68087196324). Truncating to maximum value.
  warn("Device model returned an invalid

In [6]:
qc=QuantumCircuit(1,1)
qc.h(0)
qc.measure(0,0)

def qrng(bits,sim):

    """qrng between 1 and 2^n
    n=bits (int)
    sim:bool true=simulate, false = use realqcomp"""

    bit_string=''
    for i in range(bits):
        if sim:
            job_sim = backend_sim.run(transpile(qc, backend_sim), shots=1)
            result_sim = job_sim.result()
            counts = result_sim.get_counts(qc)
            gen=(str(counts)[2]) #generated bit
            bit_string+=gen
        else:
            job=backend.run(transpile(qc,backend_sim),shots=1)
            result=job.result()
            counts=result.get_counts(qc)
            gen=(str(counts)[2])
            #{[0]:1}
            bit_string+=gen

    return(int(bit_string,2)+1)


def qrng_range(difficulties,selection,sim):

    """generates a random number between min and max inclusive
    difficulties: dict of difficulty modes
    selection: string ("easy", "medium", "hard", "extreme")
    sim:bool true=simulate, false = use realqcomp

    easy: 1-100 // 2^7
    medium:1-1000 // 2^10
    hard:1-100,000 // 2^17
    extreme:1-1,000,000 // 2^20
    """

    number=9999999
    while number>difficulties[selection][0]:
        number=qrng(difficulties[selection][1],sim)
        
    return number

def rand_num_guesser_game(sim=True):
    
    modes={} #dict mapping difficulty mode to tuple indicating (max for guessing,required #bits)
    modes["harder"]=(5,3)
    modes["medium"]=(10,4)
    modes["easy"]=(50,6)
    modes["easiest"]=(1000,10)
    modes["too easy"]=(1000000,20)

    print("let's play guess that number!\nhow it works: you and qcomp make a guess at the same time. qcomp knows your guesses, but you don't know its guesses")
    mode=str(input("try to guess the number before the quantum computer does\nchoose your difficulty (easy, medium, harder):"))
    
    random_Num = qrng_range(modes,mode,sim)
    #print(random_Num) #printing for testing
    
    print("\nguess a number between 1 and " + str(modes[mode][0]) + ":")
    
    #qcomp makes a guess
    guessed=[]
    q_guess=qrng_range(modes,mode,sim)
    guessed.append(q_guess)

    print("(qcomp is guessing...)")
    
    your_Num = int(input("your guess:"))
    guessed.append(your_Num)
    #calculates the absolute distance between the number and the guess
    distance = abs(random_Num-your_Num);
    #your previous guess
    last_distance = 999999999999;


    
    #each iteration of this loop is a guess
    qcomp_lost=True
    while your_Num != random_Num:
        
        #qcomp makes another guess
        new_guess=q_guess
        while new_guess in guessed:
            new_guess=qrng_range(modes,mode,sim)
        guessed.append(new_guess)
        
        if new_guess ==random_Num:
            print("qcomp guessed correctly-"+ str(new_guess)+". you lose!")
            qcomp_lost=False
            break
        else:
            print("(qcomp is guessing...)")
        
        
        distance = abs(random_Num-int(your_Num))
        guessed.append(your_Num)
        if your_Num == "break":
            break
        
        
        """
        if your current guess is less than your previous guess that means you are closer to the number
            if its greater than, it means you are colder, otherwise your guess its probably the same as last time
        """
        if distance < last_distance:
            print("you're getting warmer. guess again")
        elif distance > last_distance:
            print("you're getting colder. guess again")
        else:
            print("try again")
        
        

        last_distance = distance
        your_Num = int(input("your guess:"))
        
        
        
    if qcomp_lost:
        print("you win! the number was", random_Num)
    
    
def main():
    
    rand_num_guesser_game()
    #true = use simulator
    #false = use real qcomp

if __name__ == "__main__":
    main()


let's play guess that number!
how it works: you and qcomp make a guess at the same time. qcomp knows your guesses, but you don't know its guesses


try to guess the number before the quantum computer does
choose your difficulty (easy, medium, harder): medium



guess a number between 1 and 10:
(qcomp is guessing...)


your guess: 5


(qcomp is guessing...)
you're getting warmer. guess again


...

(qcomp is guessing...)
you're getting warmer. guess again


your guess: 9


(qcomp is guessing...)
you're getting warmer. guess again


your guess: 10


you win! the number was 10
